In [3]:
ticker = input('Please input the ticker of the stock: ')

In [4]:
#common parts
import requests
from bs4 import BeautifulSoup
link = 'https://finance.yahoo.com/quote/MS/financials?p=' + ticker
response = requests.get(link)
if response.status_code == 200:
    pass
else:
    print('An error occurred when accessing the page!')

In [5]:
def analysis(ticker):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    link = 'https://finance.yahoo.com/quote/' + ticker + '/analysis?p' + ticker
    response = requests.get(link)
    if response.status_code == 200:
        pass
    else:
        print('An error occurred when accessing the page!')
    result_page = BeautifulSoup(response.content,'lxml')
    df_list = []
    for table_tag in result_page.find_all('table'):
        col,data_col,data_frame = [], [], []
        for th_tag in table_tag.find_all('th'):
            col.append(th_tag.get_text())
        i = 1
        for td_tag in table_tag.find_all('td'):
            if 1 <= i <= 5:
                data_col.append(td_tag.get_text())
                i += 1
            else:
                data_frame.append(data_col)
                data_col = []
                i = 1
                data_col.append(td_tag.get_text())
                i += 1
        data_frame.append(data_col)
        df = pd.DataFrame(data_frame,columns = col)
        df.set_index(col[0],inplace=True)
        df_list.append(df)
    return df_list[0]
#     return df_list[0],df_list[1],df_list[2],df_list[3],df_list[4],df_list[5]

In [6]:
def income_statement(ticker):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    link = 'https://finance.yahoo.com/quote/' + ticker + '/financials?p=' + ticker
    response = requests.get(link)
    if response.status_code == 200:
        pass
    else:
        print('An error occurred when accessing the page!')
    result_page = BeautifulSoup(response.content,'lxml')
    label,col,data_col,data_frame = [],[],[],[]
    for tag in result_page.find_all('td')[:5]:
        col.append(tag.get_text())
    col[0] = col[0] + ' (All numbers in thousands)'
    i = 1
    for tag in result_page.find_all('td')[5:]:
        if tag.get('class') != ['Fw(b)', 'Fz(15px)', 'Pb(8px)', 'Pt(36px)']:
            if 1 <= i <= 5:
                data_col.append(tag.get_text())
                i += 1
            else:
                data_frame.append(data_col)
                data_col = []
                i = 1
                data_col.append(tag.get_text())
                i += 1
    data_frame.append(data_col)
    df = pd.DataFrame(data_frame,columns = col)
    df.set_index(col[0],inplace=True)
    return df

In [8]:
def balance_sheet(ticker):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    link = 'https://finance.yahoo.com/quote/' + ticker + '/balance-sheet?p=' + ticker
    response = requests.get(link)
    result_page = BeautifulSoup(response.content,'lxml')
    df_list = []
    col,data_col,data_frame = [], [], []
    i = 1
    end = 0
    for td_tag in result_page.find_all('td')[:5]:
        col.append(td_tag.get_text())
    col[0] = col[0] + ' (All numbers in thousands)'
    for td_tag in result_page.find_all('td')[5:]:
        if td_tag.get('class') in [['Fw(b)', 'Fz(15px)'],['C($gray)', 'Ta(end)']]:
            col.append(td_tag.get_text())
        elif td_tag.get('class') != ['Fw(b)', 'Fz(15px)', 'Pb(8px)', 'Pt(36px)']:
            if 1 <= i <= 5:
                data_col.append(td_tag.get_text())
                i += 1
            else:
                data_frame.append(data_col)
                data_col = []
                i = 1
                data_col.append(td_tag.get_text())
                i += 1
        if td_tag.get('class') == ['Fw(b)', 'Fz(s)', 'Ta(end)', 'Pb(20px)']:
            end += 1
        if end == 4:
            data_frame.append(data_col)
            df = pd.DataFrame(data_frame,columns = col)
            df.set_index(col[0],inplace=True)
            df_list.append(df)
            end = 0
            i = 1
            data_col,data_frame = [], []
    return df_list[0]

In [9]:
balance_sheet('MS')

,12/31/2017,12/31/2016,12/31/2015,12/31/2014
Period Ending (All numbers in thousands),,,,
Cash And Cash Equivalents,"24,816,000","22,017,000","19,827,000","21,381,000"
Short Term Investments,"208,268,000","227,191,000","261,542,000","260,603,000"
Net Receivables,"148,309,000","127,554,000","116,821,000","105,608,000"
Inventory,-,-,-,-
Other Current Assets,"66,752,000","67,887,000","47,456,000","35,061,000"
Total Current Assets,"747,258,000","707,413,000","686,296,000","679,926,000"
Long Term Investments,"81,425,000","82,929,000","75,127,000","73,671,000"
Property Plant and Equipment,-,-,-,"6,108,000"
Goodwill,"6,597,000","6,577,000","6,584,000","6,588,000"
